## run this cell before starting spark cluster connection (corrects module error)

In [1]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [67]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [68]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', None)

### select the data

In [69]:
df_ps = spark.read.json("/user/sprenner/training_all_april.json")

### Create temporary table view

In [70]:
from pyspark.ml.feature import Bucketizer

splits = [-float("inf"), 2, float("inf")]
bucketizer = Bucketizer(splits=splits, inputCol="hits", outputCol="bucketedFeatures")
bucketedData = bucketizer.transform(df_ps)
bucketedData.createOrReplaceTempView("bucket")

In [71]:
spark.sql("SELECT bucketedFeatures, count(*) \
            FROM bucket \
            GROUP BY  bucketedFeatures \
            ORDER BY 1 ASC").toPandas()

bucketedFeatures  count(1)
0               0.0   1690448
1               1.0   1540848

In [72]:
final_data = bucketedData.withColumn("class_bucket", when(bucketedData["bucketedFeatures"] == 0.0, "nocache").otherwise("cache"))

In [73]:
final_data.createOrReplaceTempView("final")
spark.sql("SELECT class_bucket, count(*) \
            FROM final \
            GROUP BY  class_bucket \
            ORDER BY 1 ASC").toPandas()

class_bucket  count(1)
0        cache   1540848
1      nocache   1690448

In [74]:
final_data.describe().toPandas()

summary  account           accountHash         clientState  \
0   count  3231296               3231296             3231296   
1    mean     None  -6.862163155897912E7                None   
2  stddev     None   1.214873416428219E9                None   
3     min      aad           -2144719344  CannotAuthenticate   
4     max   zzheng            2143592175         SystemError   

        clientStateHash                                            dataset  \
0               3231296                                            3231296   
1  -9.350691361360884E8                                               None   
2  2.5054777314131996E8                                               None   
3           -1922357674                           BBS_M1000_nominal_100418   
4            1269532611  valid3.110401.PowhegPythia_P2012_ttbar_nonallh...   

             datasetHash datasetScope      datasetScopeHash eventType  \
0                3231296      3231296               3231296   3231296   
1  -4.5230852327388145E7         None  -4.507730727803677E8      None   
2    1.245301801667085E9         None  1.0869769945929072E9      None   
3            -2147475923    cond09_mc           -2145215727  download   
4             2147475364       valid3            2131974379  download   

  eventTypeHash eventVersion      eventVersionHash  \
0       3231296      3231296               3231296   
1    5.937808E7         None  -1.963768093741567E9   
2           0.0         None   3.851356602973206E8   
3      59378080       1.13.2           -2044360925   
4      59378080          api            1261702107   

                           filename          filenameHash  \
0                           3231296               3231296   
1                              None   -1742584.3853425994   
2                              None  1.2397161559696252E9   
3  AOD.05371635._000001.pool.root.1           -2147481484   
4            wjets_v1.15_small.root            2147482617   

        filesize_double               hits                     hostname  \
0               3231296            3231296                      3231296   
1   3.482938720038774E8  5.085354916417438                         None   
2  1.0850033994592447E9  35.12657136873028                         None   
3                   5.0                  1  130-199-21-196.dhcp.bnl.gov   
4       3.0587790406E10               1348    xenia2.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               3231296  3231296              3231296   
1    9.92930343842449E7     None  4.964437819779924E8   
2  1.3049200474702451E9     None  9.396917290695149E8   
3           -2142788649     davs          -1607963002   
4            2143857082      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   3231296               3231296    3231296   
1                      None  3.4601683712934375E7       None   
2                      None  1.2936656207502983E9       None   
3           AGLT2_CALIBDISK           -2135244854  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid3   

              scopeHash                           traceId  \
0               3231296                           3231296   
1  -4.625174831601209E8                          Infinity   
2  1.0843476904190617E9                               NaN   
3           -2145215727  00000a90a2c841548def8b19b974a7bf   
4            2131974379  fffffa77425f474b9fa9d590bdc6e23c   

            traceIdHash                    traceIp            traceIpHash  \
0               3231296                    3231296                3231296   
1    -620846.8997167081                       None  -2.1964318650645006E8   
2  1.2399847555363288E9                       None   1.2257798827174745E9   
3           -2147483615  2001:1458:201:e3::100:107            -2143850866   
4            2147482933        ::ffff:95.103.58.33             2146432649   

     tra

In [75]:
final_data = final_data.drop("traceTimeentry")

In [76]:
categoricalFeatures = ['account', 'clientState', 'dataset', 'datasetScope', 'eventType', 'eventVersion', 'filename', 
                       'hostname', 'protocol', 'remoteSite', 'scope', 'traceId', 'traceIp', 'uuid']

encoded = final_data

for elem in categoricalFeatures:
    encoded = encoded.withColumn(elem + "Hash", hash(encoded[elem]))

In [77]:
encoded.describe().toPandas()

summary  account           accountHash         clientState  \
0   count  3231296               3231296             3231296   
1    mean     None  -6.862163155897912E7                None   
2  stddev     None   1.214873416428219E9                None   
3     min      aad           -2144719344  CannotAuthenticate   
4     max   zzheng            2143592175         SystemError   

        clientStateHash                                            dataset  \
0               3231296                                            3231296   
1  -9.350691361360884E8                                               None   
2  2.5054777314132005E8                                               None   
3           -1922357674                           BBS_M1000_nominal_100418   
4            1269532611  valid3.110401.PowhegPythia_P2012_ttbar_nonallh...   

             datasetHash datasetScope      datasetScopeHash eventType  \
0                3231296      3231296               3231296   3231296   
1  -4.5230852327388145E7         None  -4.507730727803677E8      None   
2    1.245301801667085E9         None  1.0869769945929074E9      None   
3            -2147475923    cond09_mc           -2145215727  download   
4             2147475364       valid3            2131974379  download   

  eventTypeHash eventVersion      eventVersionHash  \
0       3231296      3231296               3231296   
1    5.937808E7         None  -1.963768093741567E9   
2           0.0         None  3.8513566029732066E8   
3      59378080       1.13.2           -2044360925   
4      59378080          api            1261702107   

                           filename          filenameHash  \
0                           3231296               3231296   
1                              None   -1742584.3853425994   
2                              None  1.2397161559696255E9   
3  AOD.05371635._000001.pool.root.1           -2147481484   
4            wjets_v1.15_small.root            2147482617   

        filesize_double               hits                     hostname  \
0               3231296            3231296                      3231296   
1   3.482938720038774E8  5.085354916417438                         None   
2  1.0850033994592445E9  35.12657136873029                         None   
3                   5.0                  1  130-199-21-196.dhcp.bnl.gov   
4       3.0587790406E10               1348    xenia2.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               3231296  3231296              3231296   
1    9.92930343842449E7     None  4.964437819779924E8   
2  1.3049200474702454E9     None   9.39691729069515E8   
3           -2142788649     davs          -1607963002   
4            2143857082      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   3231296               3231296    3231296   
1                      None  3.4601683712934375E7       None   
2                      None  1.2936656207502983E9       None   
3           AGLT2_CALIBDISK           -2135244854  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid3   

              scopeHash                           traceId  \
0               3231296                           3231296   
1  -4.625174831601209E8                          Infinity   
2  1.0843476904190621E9                               NaN   
3           -2145215727  00000a90a2c841548def8b19b974a7bf   
4            2131974379  fffffa77425f474b9fa9d590bdc6e23c   

           traceIdHash                    traceIp            traceIpHash  \
0              3231296                    3231296                3231296   
1   -620846.8997167081                       None  -2.1964318650645006E8   
2  1.239984755536329E9                       None   1.2257798827174745E9   
3          -2147483615  2001:1458:201:e3::100:107            -2143850866   
4           2147482933        ::ffff:95.103.58.33             2146432649   

     traceTime

In [78]:
#predictions.write.format("json").save("/user/sprenner/forest_training_all_april.json")

## Bucketing and Hashing End

In [79]:
# TEST FOREST

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator




assembler = VectorAssembler(inputCols=["accountHash","clientStateHash","datasetHash","datasetScopeHash","eventTypeHash",
                                       "eventVersionHash","filenameHash","filesize_double","hostnameHash","protocolHash",
                                       "remoteSiteHash","scopeHash","traceIdHash","traceIpHash","traceTimeentryUnix",
                                       "transferStart","uuidHash"], outputCol="features")
output = assembler.transform(encoded)
output = output.select("bucketedFeatures", "features")
        
train_data, test_data = output.randomSplit([.8,.2],seed=1234)
rf = RandomForestClassifier(labelCol='bucketedFeatures', featuresCol='features',numTrees=300, seed=1234)
model = rf.fit(train_data)
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="bucketedFeatures", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.201792
(17,[0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16],[0.0882351038016,0.462423976244,0.0225377308703,0.0877887246922,0.0023468467851,4.41818015563e-07,0.0590341044656,0.0369847733128,0.0158489582969,0.002001079585,0.121914926186,2.42692342241e-07,0.0164987819806,0.0405166066252,0.0388641408024,0.00500356184152])


In [80]:
model.featureImportances

## predict downloads for may with trained model

In [81]:
df_mai = spark.read.json("/user/sprenner/training_all_mai.json")

In [82]:
df_mai.describe().toPandas()

summary   account           accountHash     clientState  \
0   count   5316524               5316524         5316524   
1    mean      None   5.676034578304038E7            None   
2  stddev      None  1.0782381847434077E9            None   
3     min  aalfonsi           -2144719344  AttributeError   
4     max    zzheng            2143592175     SystemError   

        clientStateHash                                            dataset  \
0               5316524                                            5316524   
1  -8.556539364422513E8                                               None   
2   3.393227998953656E8                                               None   
3           -1913538301                                    13917493.events   
4            1269532611  valid2.361023.Pythia8EvtGen_A14NNPDF23LO_jetje...   

           datasetHash datasetScope      datasetScopeHash eventType  \
0              5316524      5316524               5316524   5316524   
1  8.513416665722266E7         None  -2.489378879578484E8      None   
2  1.264352081440324E9         None   1.159227636261917E9      None   
3          -2147477854    cond09_mc           -2145215727  download   
4           2147445885       valid2            2144116684  download   

  eventTypeHash eventVersion       eventVersionHash  \
0       5316524      5316524                5316524   
1    5.937808E7         None  -2.0386347870917916E9   
2           0.0         None   1.2667515763379405E8   
3      59378080       1.13.2            -2044360925   
4      59378080       1.16.1             1352846300   

                                            filename         filenameHash  \
0                                            5316524              5316524   
1                                               None   -576003.5649204254   
2                                               None  1.238741621683768E9   
3                                              0.dat          -2147483353   
4  user.zzheng.410389.MadGraphPythia8EvtGen.DAOD_...           2147479805   

        filesize_double                hits                  hostname  \
0               5316524             5316524                   5316524   
1  2.8167336660135007E8   6.517046664324284                      None   
2  1.0094883391407572E9  13.920048576048558                      None   
3                   2.0                   1    ReCaS-ui-01.cs.infn.it   
4       4.7520668166E10                 249  xenia.nevis.columbia.edu   

           hostnameHash protocol         protocolHash  \
0               5316524  5316524              5316524   
1  1.8503396854168174E8     None  5.671483465013922E8   
2  1.1133988987325292E9     None  9.234807216961868E8   
3           -2140594599     davs          -1607963002   
4            2130161317      srm           2128034789   

                 remoteSite        remoteSiteHash      scope  \
0                   5316524               5316524    5316524   
1                      None  2.0771631249619377E8       None   
2                      None   1.243718578580587E9       None   
3           AGLT2_CALIBDISK           -2113133296  cond09_mc   
4  ZA-WITS-CORE_SCRATCHDISK            2141740830     valid2   

               scopeHash                           traceId  \
0                5316524                           5316524   
1  -2.4953820189199993E8                          Infinity   
2      1.1593529202301E9                               NaN   
3            -2145215727  00000696ec754c998f70d84d737dfd18   
4             2144116684  fffffe65b9c549e5ae746c15ec98d5bb   

            traceIdHash                    traceIp            traceIpHash  \
0               5316524                    5316524                5316524   
1   -254286.60280984343                       None  -4.1963757976758495E7   
2  1.2400714422504258E9                       None   1.2010453269587853E9   
3           -2147483583  2001:1458:201:e4::100:28c            -2143850866   
4            2147482

In [83]:
assembler_mai = VectorAssembler(
    inputCols=["traceTimeentryUnix", "transferStart", "accountHash","clientStateHash","datasetHash", "datasetScopeHash",
               "eventTypeHash", "eventVersionHash","filenameHash", "hostnameHash", "protocolHash", "remoteSiteHash", 
               "scopeHash", "traceIdHash", "traceIpHash", "uuidHash", "filesize_double"],
    outputCol="features")

#linearRegression will look for a column called features
output_mai = assembler_mai.transform(df_mai)

In [84]:
predicted_mai = model.transform(output_mai)

In [85]:
predicted_mai=predicted_mai.withColumn("filesize", predicted_mai["filesize_double"].cast(IntegerType()))

In [86]:
predicted_mai = predicted_mai.drop('filesize_double')

## save separately for plotting

In [87]:
df_mai_plot = predicted_mai.select('hits', 'prediction')
df_mai_plot.write.csv('/user/sprenner/forest_plot_mai_all_predictions_numTrees100_seed1234_cache01.csv', sep='\t')

In [88]:
df_mai_sim = predicted_mai.select('uuid', 'account', 'dataset', 'filename', 'eventType', 'clientState', 
                                  from_unixtime(predicted_mai.traceTimeentryUnix, format='yyyy-MM-dd').alias('day'), 'traceTimeentryUnix', 
                                  'filesize', 'prediction').sort(col("day").asc())

In [89]:
df_mai_sim.limit(5).toPandas()

uuid account  \
0  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   
1  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   
2  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   
3  a71a723e9f9341c5a18bbdc09b4ee87e   cylin   
4  c2bb4c50d50f474d82b8eef0a4aaf02a   cylin   

                                             dataset  \
0  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   
1  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   
2  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   
3  mc16_13TeV.361021.Pythia8EvtGen_A14NNPDF23LO_j...   
4  mc16_13TeV.361024.Pythia8EvtGen_A14NNPDF23LO_j...   

                           filename eventType         clientState         day  \
0  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   
1  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   
2  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   
3  AOD.12911492._000332.pool.root.1  download  ServiceUnavailable  2018-05-01   
4  AOD.12911516._000058.pool.root.1  download  ServiceUnavailable  2018-05-01   

   traceTimeentryUnix    filesize  prediction  
0        1.525170e+09  2147483647         1.0  
1        1.525170e+09  2147483647         1.0  
2        1.525170e+09  2147483647         1.0  
3        1.525170e+09  2147483647         1.0  
4        1.525170e+09  2147483647         1.0

In [90]:
df_mai_sim.write.csv('/user/sprenner/forest_mai_all_predictions_numTrees100_seed1234_cache01.csv', sep='\t')